In [12]:
from functools import partial

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from datasets import load_dataset

from graph import Graph
from dataset import EAPDataset
from attribute import attribute
from metrics import get_metric
from evaluate_graph import evaluate_graph, evaluate_baseline, evaluate_area_under_curve

In [13]:
model = HookedTransformer.from_pretrained("gpt2-small", device="cuda")
model.cfg.use_split_qkv_input = True
model.cfg.use_attn_result = True
model.cfg.use_hook_mlp_in = True

Loaded pretrained model gpt2-small into HookedTransformer


In [14]:
dataset = EAPDataset("greater-than-gpt2.csv")
dataset.head(500)
dataloader = dataset.to_dataloader(64)
metric_fn = get_metric("prob_diff", "greater-than", model.tokenizer, model)

This cell uses activation-space EAP-IG with 5 steps to attribute using patching

In [15]:
g = Graph.from_model(model)
attribute(model, g, dataloader, partial(metric_fn, loss=True, mean=True), method='EAP-IG-activations', ig_steps=5)
g.apply_greedy(250)
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=True).mean().item()
baseline = evaluate_baseline(model, dataloader, partial(metric_fn, loss=False, mean=False)).mean().item()
print(f"Faithfulness: {results / baseline}")

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:00<00:00, 14.19it/s]

Faithfulness: 0.7839652423889335


Same thing, but EAP-IG in input space, which strangely enough does better (normally they're about the same)

In [16]:
g2 = Graph.from_model(model)
attribute(model, g2, dataloader, partial(metric_fn, loss=True, mean=True), method='EAP-IG-inputs', ig_steps=5)
g2.apply_greedy(250)
results = evaluate_graph(model, g2, dataloader, partial(metric_fn, loss=False, mean=False), prune=True).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:01<00:00,  6.13it/s]

Faithfulness: 0.9261200085790293


What about if we apply zero ablations to these graphs?

In [17]:
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='zero').mean().item()
print(f"Faithfulness: {results / baseline}")

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:01<00:00,  7.04it/s]

Faithfulness: -2.045945751205287e-05


And if we use an attribution method that natively uses zero gradients? Also consider trying with more ig_steps!

In [19]:
g3 = Graph.from_model(model)
attribute(model, g3, dataloader, partial(metric_fn, loss=True, mean=True), method='EAP-IG-activations', intervention='zero', ig_steps=5)
g3.apply_greedy(250)
results = evaluate_graph(model, g3, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='zero').mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:00<00:00,  9.01it/s]

Faithfulness: -0.0007790679122014042


What about the reverse—zero attribution but patching evaluation?

In [ ]:
g3.apply_greedy(250)
results = evaluate_graph(model, g3, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='zero').mean().item()
baseline = evaluate_baseline(model, dataloader, partial(metric_fn, loss=False, mean=False)).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:00<00:00, 14.11it/s]

Faithfulness: 0.33430727842985763


Now, let's do some mean ablations! Here's a generic dataset:

In [20]:
mean_dataset = load_dataset('stas/openwebtext-10k', split='train')
intervention_dataloader = DataLoader(mean_dataset['text'][:1000], batch_size=16)

Let's do mean ablations on a graph scored with patching ablations

In [30]:
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='mean', intervention_dataloader=intervention_dataloader).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:01<00:00,  7.04it/s]

Faithfulness: -2.112404321439537e-05


And now on a graph scored with mean ablations

In [22]:
g4 = Graph.from_model(model)
attribute(model, g4, dataloader, partial(metric_fn, loss=True, mean=True), method='EAP-IG-activations', intervention='mean', ig_steps=5, intervention_dataloader=intervention_dataloader)
g4.apply_greedy(250)
results = evaluate_graph(model, g4, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='mean', intervention_dataloader=intervention_dataloader).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:00<00:00,  8.74it/s]

Faithfulness: -0.0007573959612862081


What if we do positional ablations, which preserve task information? We can do this using either the original dataset (but note that in the greater-than task, this leads to positional mean ablations being a less-destructive ablation than patching) or just the corrupted version of the dataset. Let's start with the original:

In [28]:
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='mean-positional', intervention_dataloader=dataloader).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:01<00:00,  7.10it/s]

Faithfulness: 0.9017045250395437


In [24]:
g5 = Graph.from_model(model)
attribute(model, g5, dataloader, partial(metric_fn, loss=True, mean=True), method='EAP-IG-activations', intervention='mean-positional', ig_steps=5, intervention_dataloader=dataloader)
g5.apply_greedy(250)
results = evaluate_graph(model, g5, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='mean-positional', intervention_dataloader=dataloader).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:00<00:00,  8.00it/s]

Faithfulness: 0.9145310149099368


And then try the corrupted:

In [25]:
from copy import deepcopy
corrupted_dataset = deepcopy(dataset)
corrupted_dataset.df['clean'] = corrupted_dataset.df['corrupted']
corrupted_dataloader = corrupted_dataset.to_dataloader(16)

In [29]:
results = evaluate_graph(model, g, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='mean-positional', intervention_dataloader=corrupted_dataloader).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:01<00:00,  7.05it/s]

Faithfulness: 0.7456998058356434


In [27]:
g6 = Graph.from_model(model)
attribute(model, g6, dataloader, partial(metric_fn, loss=True, mean=True), method='EAP-IG-activations', intervention='mean-positional', ig_steps=5, intervention_dataloader=corrupted_dataloader)
g6.apply_greedy(250)
results = evaluate_graph(model, g6, dataloader, partial(metric_fn, loss=False, mean=False), prune=True, intervention='mean-positional', intervention_dataloader=corrupted_dataloader).mean().item()
print(f"Faithfulness: {results / baseline}")

100%|██████████| 8/8 [00:01<00:00,  7.56it/s]

Faithfulness: 0.7835840997563741
